In [1]:
%load_ext autoreload
%autoreload 2

from copy import deepcopy
from collections import OrderedDict
import cv2
import copy
import os
import matplotlib
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import scipy as sp
import numpy as np
import point_cloud_utils as pcu
import torch
import torch.nn.functional as F
from skimage.transform import resize
from skimage.color import rgb2gray
from tqdm.notebook import tqdm as tqdm

import torch
import torchvision

import dense_correspondence_manipulation.utils.utils as utils

import meshcat
import meshcat.geometry as g

import detectron2.utils.comm as comm
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.config import get_cfg
from detectron2.config import CfgNode as CN
from detectron2.data import (
    MetadataCatalog,
    build_detection_test_loader,
    build_detection_train_loader,
)
from detectron2.data.datasets import load_coco_json
from detectron2.engine import DefaultTrainer, DefaultPredictor, default_argument_parser, default_setup, launch
from detectron2.evaluation import inference_on_dataset, COCOEvaluator
from detectron2.modeling import build_model
from detectron2.utils.events import EventStorage
from detectron2.utils.logger import setup_logger
from detectron2.utils.visualizer import Visualizer

import scene_generation.inverse_graphics.synthetic_scene_database_loader as loader
from scene_generation.inverse_graphics.synthetic_scene_database_loader import XenRCNNMapper
import scene_generation.inverse_graphics.keypoint_mcmc.roi_heads as roi_heads
from scene_generation.inverse_graphics.keypoint_mcmc.particle_filter_icp import *

from pydrake.math import RigidTransform, RotationMatrix, RollPitchYaw
from pydrake.solvers.mathematicalprogram import MathematicalProgram, Solve

np.set_printoptions(precision=2)
torch.set_default_tensor_type(torch.FloatTensor)

DATA_ROOT = "/home/gizatt/data/generated_cardboard_envs/"
DETECTRON_ROOT = "/home/gizatt/tools/detectron2/"


class InstanceCloud():
    def __init__(self, pts, colors, descriptors):
        self.pts = pts
        self.descriptors = descriptors
        self.colors = colors
    def get_augmented_pts(self, descriptor_factor=1.):
        return np.vstack([self.pts, descriptor_factor*self.descriptors])
    
%matplotlib inline
def cv2_imshow(im):
    plt.imshow(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))
    plt.axis('off')

print(torchvision.__file__)

load_dict = torch.load("run_on_all_records.pt")
all_instance_clouds_by_record = load_dict["all_instance_clouds_by_record"]
affinities_by_record = load_dict["affinities_by_record"]
clusters_by_record = load_dict["clusters_by_record"]

/home/gizatt/miniconda3/envs/py36_pyro/lib/python3.6/site-packages/torchvision-0.6.0a0+fb562f5-py3.6-linux-x86_64.egg/torchvision/__init__.py


In [2]:
# Load in data into numpy and o3d structures.

import open3d as o3d
import copy
import matplotlib.cm as cm
import numpy as np

clusters = []
for c in list(clusters_by_record.values())[:10]:
    clusters += c
cluster = clusters[3]
    
vis = meshcat.Visualizer(zmq_url="tcp://127.0.0.1:6000")["ransac_to_descriptors"]
vis.delete()

def compute_unit_box_descriptors(pts, normals):
    # Descriptor is the viridis color coding
    # of the distance to the center of each pt's face.
    face_local = np.abs(pts)*(normals == 0.0)*2
    dist = np.linalg.norm(face_local, axis=0)/1.414
    return torch.tensor(cm.get_cmap('viridis')(dist).astype(np.float32).T[:3, :])
    
# Make canonical box with descriptors
N_model_pts = 1000
model_pts, model_normals = make_unit_box_pts_and_normals(N=N_model_pts)
model_colors = np.zeros((3, model_pts.shape[1]))
model_colors[0, :] = 1.
model_descriptors = compute_unit_box_descriptors(model_pts, model_normals).numpy()
model_pts = np.dot(np.diag([0.5, 0.4, 0.3]), model_pts.numpy())
model_normals = model_normals.numpy()

scene_pts = cluster.pts.numpy()
scene_descriptors = cluster.descriptors.numpy()

def np_to_o3d(pts, colors, normals=None):
    pts_o3d = o3d.geometry.PointCloud()
    pts_o3d.points = o3d.utility.Vector3dVector(pts.copy().T)
    pts_o3d.colors = o3d.utility.Vector3dVector(colors.copy().T)
    if normals is not None:
        pts_o3d.normals = o3d.utility.Vector3dVector(normals.copy().T)
    else:
        pts_o3d.estimate_normals()
    return pts_o3d
scene_pts_o3d = np_to_o3d(scene_pts, scene_descriptors)
model_pts_o3d = np_to_o3d(model_pts, model_descriptors, model_normals)

# Clean up the input data
scene_pts_o3d, ind = scene_pts_o3d.remove_statistical_outlier(nb_neighbors=40,
                                                              std_ratio=1.0)
## And downsample some
N_scene_pts_before = np.asarray(scene_pts_o3d.points).shape[0]
scene_pts_o3d = scene_pts_o3d.voxel_down_sample(0.01)
N_scene_pts_after = np.asarray(scene_pts_o3d.points).shape[0]
print("Downsampled scene from %d pts to %d pts." % (N_scene_pts_before, N_scene_pts_after))
# Pre-generate kd trees that might be in use
scene_kdtree = o3d.geometry.KDTreeFlann(scene_pts_o3d)
model_kdtree = o3d.geometry.KDTreeFlann(model_pts_o3d)

vis["scene"].set_object(
    g.PointCloud(position=np.asarray(scene_pts_o3d.points).T,
                 color=np.asarray(scene_pts_o3d.colors).T,
                 size=0.01))

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/
Downsampled scene from 31834 pts to 4861 pts.


In [60]:
def select_ransac_inlier_set(N_corrs_to_select = 5):
    # Pick N from the scene, and pick randomly from among the
    # points in each of their neighborhood
    scene_pt_ransac = []
    model_pt_ransac = []
    model_inds_for_masking = np.array(range(N_model_pts))
    while len(scene_pt_ransac) < N_corrs_to_select:
        scene_pt_ind = np.random.choice(N_scene_pts_after)
        scene_pt = scene_pts_o3d.points[scene_pt_ind]
        target_color = scene_pts_o3d.colors[scene_pt_ind]
        model_pt_mask = np.linalg.norm(model_descriptors.T - target_color, axis=1) < 0.05
        if np.sum(model_pt_mask) > 0:
            model_pt_ind = np.random.choice(model_inds_for_masking[model_pt_mask])
            scene_pt_ransac.append(scene_pt)
            model_pt_ransac.append(model_pts[:, model_pt_ind])
    model_pt_ransac = np.stack(model_pt_ransac)
    scene_pt_ransac = np.stack(scene_pt_ransac)
    return model_pt_ransac, scene_pt_ransac

def fit_R_t_s(model_pt_ransac, scene_pt_ransac):
    # Fit R, T, and scaling to the inlier set
    prog = MathematicalProgram()
    R = prog.NewContinuousVariables(3, 3, "R")
    t = prog.NewContinuousVariables(3, "t")
    s = prog.NewContinuousVariables(3, "s")
    prog.AddBoundingBoxConstraint(np.ones(3)*0.1, np.ones(3)*2., s)
    prog.AddBoundingBoxConstraint(np.ones(3)*-5., np.ones(3)*5., t)
    prog.AddBoundingBoxConstraint(np.ones(9)*-1., np.ones(9)*1., R.flatten())
    slack = prog.NewContinuousVariables(1, "slack")[0]
    lhs = R.T.dot(R)
    rhs = np.eye(3)
    for u in range(3):
        for v in range(3):
            prog.AddConstraint(rhs[u, v] - lhs[u, v] >= -slack)
            prog.AddConstraint(rhs[u, v] - lhs[u, v] <= slack)
    #prog.AddConstraint(slack <= 0.1)
    for model_pt, scene_pt in zip(model_pt_ransac, scene_pt_ransac):
        err = scene_pt - (np.dot(R, np.diag(s).dot(model_pt)) + t)
        prog.AddCost(np.sum(err**2.))
    prog.AddCost(slack ** 2.)
    prog.SetInitialGuess(R, np.eye(3))
    result = Solve(prog)
    if result.is_success():
        R = result.GetSolution(R)
        t = result.GetSolution(t)
        s = result.GetSolution(s)
        return True, R, t, s
    else:
        return False, None, None, None
    
def fit_Rs_t(model_pt_ransac, scene_pt_ransac):
    # Fit R*s and T to the inlier set
    prog = MathematicalProgram()
    Rs = prog.NewContinuousVariables(3, 3, "Rs")
    t = prog.NewContinuousVariables(3, "t")
    prog.AddBoundingBoxConstraint(np.ones(3)*-5., np.ones(3)*5., t)
    prog.AddBoundingBoxConstraint(np.ones(9)*-10., np.ones(9)*10., Rs.flatten())
    for model_pt, scene_pt in zip(model_pt_ransac, scene_pt_ransac):
        err = scene_pt - (Rs.dot(model_pt.T) + t)
        prog.AddCost(np.sum(err**2.))
    prog.SetInitialGuess(Rs, np.eye(3))
    result = Solve(prog)
    if result.is_success():
        Rs = result.GetSolution(Rs)
        t = result.GetSolution(t)
        return True, Rs, t
    else:
        return False, None, None, None
    
best_fitness = 0.
best_tf = np.eye(4)

for k in tqdm(range(1000)):
    model_pt_ransac, scene_pt_ransac = select_ransac_inlier_set(5)
    
    # This version doesn't always succeed (usually doesn't, actually)
    # at getting the global min. Maybe the R^T R = I constraint is confusing
    # the solver?
    #success, R, t, s = fit_R_t_s(model_pt_ransac, scene_pt_ransac)
    #Rs = np.dot(R, np.diag(s))
    success, Rs, t = fit_Rs_t(model_pt_ransac, scene_pt_ransac)
    Rs = Rs
    U, S, V = np.linalg.svd(Rs)
    R = np.dot(U, V.T)
    # Find scale + skew SS s.t. R SS = Rs
    SS = R.T.dot(Rs)
    #Rs = np.dot(R, np.diag(s))
    transformed_model_pts_ransac = (np.dot(Rs, model_pt_ransac.T).T + t).T
    vis["model_pts_ransac"].set_object(
        g.PointCloud(position=transformed_model_pts_ransac,
                     color=np.array([[1., 0., 0.]]*5).T,
                     size=0.025))
    vis["scene_pts_ransac"].set_object(
        g.PointCloud(position=scene_pt_ransac.T,
                     color=np.array([[0., 0., 1.]]*5).T,
                     size=0.025))
    
    if success:
        # Apply transformation to model points and get inlier set
        # by point distance
        tf = np.eye(4)
        tf[:3, :3] = Rs
        tf[:3, 3] = t
        results = o3d.registration.evaluate_registration(
            source=model_pts_o3d,
            target=scene_pts_o3d,
            max_correspondence_distance=0.01,
            transformation=tf)

        transformed_model_pts = (tf[:3, :3].dot(model_pts).T + tf[:3, 3]).T
        corrs = np.asarray(results.correspondence_set)
        distances = np.linalg.norm(
            transformed_model_pts[:, corrs[:, 0]] - 
            np.asarray(scene_pts_o3d.points).T[:, corrs[:, 1]], axis=0)
        fitness = np.sum(distances < 0.1) / N_scene_pts_after
        if fitness > best_fitness:
            best_fitness = fitness
            print("New best fitness: ", best_fitness)
            best_tf = tf
    else:
        print("Fitting failed")

# Drawing
transformed_model_pts = (best_tf[:3, :3].dot(model_pts).T + best_tf[:3, 3]).T
print(best_tf, best_fitness)
# Just draw
vis["transformed_model"].set_object(
    g.PointCloud(position=transformed_model_pts,
                 color=np.asarray(model_pts_o3d.colors).T,
                 size=0.01))

New best fitness:  0.01501748611396832
New best fitness:  0.1489405472125077
New best fitness:  0.1553178358362477
New best fitness:  0.18391277514914628
New best fitness:  0.20571898786257972

[[-8.91e-04  8.05e-04 -4.99e-04  3.13e-01]
 [ 4.36e-02 -1.07e-01  8.69e-02  7.42e-02]
 [ 2.70e-01  2.05e-01 -1.61e-01  2.83e-01]
 [ 0.00e+00  0.00e+00  0.00e+00  1.00e+00]] 0.20571898786257972


In [14]:
# Animate drawing correspondences with this scene cloud
scene_offset = -np.mean(cluster.pts.numpy(), axis=1) + np.array([1., 0., 0.])
vis["scene"].set_object(
    g.PointCloud(position=(cluster.pts.numpy().T + scene_offset).T,
                 color=cluster.colors.numpy(),
                 size=0.005))

for x in np.linspace(0., 1., 100):
    target_color = cm.get_cmap('viridis')(x)[:3]
    model_inds = np.linalg.norm(model_descriptors.T - target_color, axis=1) < 0.05
    scene_inds = np.linalg.norm(scene_descriptors.T - target_color, axis=1) < 0.05
    vis["scene_highlight"].set_object(
        g.PointCloud(position=(scene_pts[:, scene_inds].T + scene_offset).T,
                     color=scene_descriptors[:, scene_inds],
                     size=0.02))
    vis["model"].set_object(
    g.PointCloud(position=model_pts[:, model_inds],
                 color=model_descriptors[:, model_inds],
                 size=0.02))
    time.sleep(0.01)